In [45]:
import carla
import numpy as np
from car_dreamer.toolkit.planner.agents.navigation.global_route_planner import GlobalRoutePlanner
from car_dreamer.toolkit.planner.agents.navigation.basic_agent import BasicAgent
from car_dreamer.toolkit.carla_manager import WorldManager

from navigator.wp_utils import xyz_to_right_lane
from navigator.visualize import CarlaVisualizer
from navigator.tsp_solver import optimize_route_order
from navigator.route_generator import generate_full_route
from navigator.test import spawn_traffic

from car_dreamer.toolkit import Config
from car_dreamer import create_task
from dreamerv3.agent import Agent as DreamerV3Agent
from car_dreamer.toolkit.observer.observer import Observer  
from car_dreamer import load_task_configs
from dreamerv3 import embodied
from dreamerv3.embodied.core import Driver

import os  
# import torch  
from car_dreamer.toolkit.observer.observer import Observer  
from car_dreamer.toolkit.observer.handlers import BirdeyeHandler  
from dreamerv3 import embodied  

In [ ]:
client = carla.Client("localhost", 2000)
client.set_timeout(20)
world = client.get_world()
carla_map = world.get_map()
ego_vehicle = next(v for v in world.get_actors().filter("vehicle.*") if v.attributes.get("role_name") == "ego_vehicle")

spawn_points = carla_map.get_spawn_points()
target_points = [
    [280.363739, 133.306351, 0.001746],
    [334.949799, 161.106171, 0.001736],
    [339.100037, 258.568939, 0.001679],
    [396.295319, 183.195740, 0.001678],
    [267.657074, 1.983160, 0.001678],
    [153.868896, 26.115866, 0.001678],
    [290.515564, 56.175072, 0.001677],
    [92.325722, 86.063644, 0.001677],
    [88.384346, 287.468567, 0.001728],
    [177.594101, 326.386902, 0.001677],
    [-1.646942, 197.501282, 0.001555],
    [59.701321, 1.970804, 0.001467],
    [122.100121, 55.142044, 0.001596],
    [161.030975, 129.313187, 0.001679],
    [184.758713, 199.424271, 0.001680],
]
sampling_resolution = 1.0

target_locations = xyz_to_right_lane(target_points, carla_map)

grp = GlobalRoutePlanner(carla_map, sampling_resolution)

optimized_targets = optimize_route_order(
    ego_vehicle.get_location(),
    target_locations,
    grp,
)

full_route = generate_full_route(ego_vehicle.get_location(), optimized_targets, grp)

Calculating pairwise distances...


In [ ]:
observer_cfg = load_task_configs('carla_navigation').env.observation

# Create a world manager wrapper for your existing world
class SimpleWorldManager:
    def __init__(self, world, ego_vehicle):
        self._world = world
        self.ego_vehicle = ego_vehicle

    def get_ego_vehicle(self):
        return self.ego_vehicle

    def carla_actors(self, actor_type=""):
        return self._world.get_actors().filter(actor_type)

    def _get_world(self):
        return self._world


# Create a simple world manager
world_manager = SimpleWorldManager(world, ego_vehicle)

# Create an observer
observer = Observer(world_manager, observer_cfg)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:21                                                                                   │
│                                                                                                  │
│   18 world_manager = SimpleWorldManager(world, ego_vehicle)                                      │
│   19                                                                                             │
│   20 # Create an observer                                                                        │
│ ❱ 21 observer = Observer(world_manager, observer_cfg)                                            │
│   22                                                                                             │
│                                                                                                  │
│ /home/zeyadcode_jammy/.local/lib/python3.10/site-packages/car_dreamer/toolkit/observer/observer. │
│ py:25 in __init__                                                                                │
│                                                                                                  │
│   22 │   def __init__(self, world: WorldManager, obs_config: dict):                              │
│   23 │   │   self._world = world                                                                 │
│   24 │   │   self._obs_config = obs_config                                                       │
│ ❱ 25 │   │   self._data_handlers = self._init_data_handlers()                                    │
│   26 │                                                                                           │
│   27 │   def register_simple_handler(                                                            │
│   28 │   │   self,                                                                               │
│                                                                                                  │
│ /home/zeyadcode_jammy/.local/lib/python3.10/site-packages/car_dreamer/toolkit/observer/observer. │
│ py:60 in _init_data_handlers                                                                     │
│                                                                                                  │
│   57 │   │   for name in self._obs_config.enabled:                                               │
│   58 │   │   │   config = self._obs_config[name]                                                 │
│   59 │   │   │   handler_class = HANDLER_DICT.get(HandlerType(config.handler))                   │
│ ❱ 60 │   │   │   handler = handler_class(self._world, config)                                    │
│   61 │   │   │   handlers[name] = handler                                                        │
│   62 │   │   return handlers                                                                     │
│   63                                                                                             │
│                                                                                                  │
│ /home/zeyadcode_jammy/.local/lib/python3.10/site-packages/car_dreamer/toolkit/observer/handlers/ │
│ sensor_handlers.py:19 in __init__                                                                │
│                                                                                                  │
│    16 │                                                                                          │
│    17 │   def __init__(self, world: WorldManager, config):                                       │
│    18 │   │   super().__init__(world, config)                                                    │
│ ❱  19 │   │   blueprint = self._world.get_blueprint(config.blueprint)                            │
│    20 │   │   if "transform" in config:                                                          │
│    21 │   │   │   self._transform = carla.Transform(carla.L

In [ ]:
  
# Load the pretrained DreamerV3 model  
checkpoint_path = "./navigation.ckpt"  # Path to your downloaded checkpoint  
  
# Create DreamerV3 config  
config = embodied.Config()  
config = config.update({  
    "logdir": "./logdir/eval",  
    "jax": {  
        "policy_devices": [0],  
        "train_devices": [0]  
    },  
    "encoder": {  
        "cnn_keys": "birdeye_wpt"  
    },  
    "decoder": {  
        "cnn_keys": "birdeye_wpt"  
    }  
})  

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/zeyadcode_jammy/.local/lib/python3.10/site-packages/car_dreamer/toolkit/config.py:57 in    │
│ __getattr__                                                                                      │
│                                                                                                  │
│    54 │   │   if name.startswith("_"):                                                           │
│    55 │   │   │   return super().__getattr__(name)                                               │
│    56 │   │   try:                                                                               │
│ ❱  57 │   │   │   return self[name]                                                              │
│    58 │   │   except KeyError:                                                                   │
│    59 │   │   │   raise AttributeError(name)                                                     │
│    60                                                                                            │
│                                                                                                  │
│ /home/zeyadcode_jammy/.local/lib/python3.10/site-packages/car_dreamer/toolkit/config.py:65 in    │
│ __getitem__                                                                                      │
│                                                                                                  │
│    62 │   │   result = self._nested                                                              │
│    63 │   │   for part in name.split(self.SEP):                                                  │
│    64 │   │   │   try:                                                                           │
│ ❱  65 │   │   │   │   result = result[part]                                                      │
│    66 │   │   │   except TypeError:                                                              │
│    67 │   │   │   │   raise KeyError                                                             │
│    68 │   │   if isinstance(result, dict):                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'shape'

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 # Create an observer                                                                        │
│ ❱  2 observer = Observer(world_manager, observer_config)                                         │
│    3                                                                                             │
│    4 # Load the pretrained DreamerV3 model                                                       │
│    5 checkpoint_path = "./navigation.ckpt"  # Path to your downloaded checkpoint                 │
│                                                                                                  │
│ /home/zeyadcode_jammy/.local/lib/python3.10/site-packages/car_dreamer/toolkit/observer/observer. │
│ py:25 in __init__                                                                                │
│                                                                                                  │
│   22 │   def __init__(self, world: WorldManager, obs_config: dict):                              │
│   23 │   │   self._world = world                                                                 │
│   24 │   │   self._obs_config = obs_config                                                       │
│ ❱ 25 │   │   self._data_handlers = self._init_data_handlers()           

In [41]:
observer = Observer(world_manager, observer_config)  

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/zeyadcode_jammy/.local/lib/python3.10/site-packages/car_dreamer/toolkit/config.py:57 in    │
│ __getattr__                                                                                      │
│                                                                                                  │
│    54 │   │   if name.startswith("_"):                                                           │
│    55 │   │   │   return super().__getattr__(name)                                               │
│    56 │   │   try:                                                                               │
│ ❱  57 │   │   │   return self[name]                                                              │
│    58 │   │   except KeyError:                                                                   │
│    59 │   │   │   raise AttributeError(name)                                                     │
│    60                                                                                            │
│                                                                                                  │
│ /home/zeyadcode_jammy/.local/lib/python3.10/site-packages/car_dreamer/toolkit/config.py:65 in    │
│ __getitem__                                                                                      │
│                                                                                                  │
│    62 │   │   result = self._nested                                                              │
│    63 │   │   for part in name.split(self.SEP):                                                  │
│    64 │   │   │   try:                                                                           │
│ ❱  65 │   │   │   │   result = result[part]                                                      │
│    66 │   │   │   except TypeError:                                                              │
│    67 │   │   │   │   raise KeyError                                                             │
│    68 │   │   if isinstance(result, dict):                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'shape'

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 observer = Observer(world_manager, observer_config)                                          │
│   2                                                                                              │
│                                                                                                  │
│ /home/zeyadcode_jammy/.local/lib/python3.10/site-packages/car_dreamer/toolkit/observer/observer. │
│ py:25 in __init__                                                                                │
│                                                                                                  │
│   22 │   def __init__(self, world: WorldManager, obs_config: dict):                              │
│   23 │   │   self._world = world                                                                 │
│   24 │   │   self._obs_config = obs_config                                                       │
│ ❱ 25 │   │   self._data_handlers = self._init_data_handlers()                                    │
│   26 │                                                                                           │
│   27 │   def register_simple_handler(                                                            │
│   28 │   │   self,                                                      

In [34]:
# Load the pretrained DreamerV3 model  
checkpoint_path = "./navigation.ckpt"  # Path to your downloaded checkpoint  
  
# Create DreamerV3 config  
config = embodied.Config()  
config = config.update({  
    "logdir": "./logdir/eval",  
    "jax": {  
        "policy_devices": [0],  
        "train_devices": [0]  
    },  
    "encoder": {  
        "cnn_keys": "birdeye_wpt"  
    },  
    "decoder": {  
        "cnn_keys": "birdeye_wpt"  
    }  
})  
  
# Create DreamerV3 agent  
agent = embodied.make_agent(config)  
agent.load(checkpoint_path)  

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:21                                                                                   │
│                                                                                                  │
│   18 })                                                                                          │
│   19                                                                                             │
│   20 # Create DreamerV3 agent                                                                    │
│ ❱ 21 agent = embodied.make_agent(config)                                                         │
│   22 agent.load(checkpoint_path)                                                                 │
│   23                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: module 'dreamerv3.embodied' has no attribute 'make_agent'